In [1]:
from label_studio_sdk import Client

# update these variables based on what you see in the terminal when starting label studio
LABEL_STUDIO_URL = 'http://localhost:8883'
API_KEY = '061663eb1d5f2e6ff2fb830326e7173c538c4817'

In [16]:
target_project = 'Testimport_rois_20221102_102108'

predictions_to_import = [
    ('20220805/20220805_0800/20220805_080714.664.0.png', 'Noctiluca'),
    ('20220623/20220623_0700/20220623_071103.834.1.png', 'Copepods')
]

In [3]:
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

In [4]:
all_projects = ls.list_projects()

In [5]:
project_names = [p.get_params()['title'] for p in all_projects]
print(project_names)

['Testimport_rois_20221102_102108', 'Phytodive_CNN_eval', 'Helgoland_2021_2', 'Phytodive_CPICS']


In [6]:
project_idx = project_names.index(target_project)

In [7]:
project = all_projects[project_idx]

In [8]:
tasks = project.get_tasks()  # for large projects, might need to specify a filter here or it will be slow

In [9]:
images = [task['data']['image'] for task in tasks]

# remove server info so we just have the relative filename
for j, s in enumerate(images):
    if ':' not in s:
        continue
    last_colon = max(loc for loc, val in enumerate(s) if val == ':')
    images[j] = s[last_colon + 6:]
    
print(images)

['20220805/20220805_0800/20220805_080714.664.0.png', '20220609/20220609_1000/20220609_102524.052.0.png', '20220623/20220623_0700/20220623_071103.834.1.png', '20220623/20220623_0700/20220623_071103.834.1.png']


In [21]:
predictions = []
for i, (img, label) in enumerate(predictions_to_import):
    task_idx = images.index(img)
    predictions.append({
        "task": tasks[task_idx]['id'],
        "result": label
    })

In [24]:
predictions

[{'task': 1091170, 'result': 'Noctiluca'},
 {'task': 1091172, 'result': 'Copepods'}]

In [23]:
project.create_predictions(predictions)

{'created': 2}